In [8]:
import random
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import copy
from torch_geometric.utils import degree
import torch
from torch import nn, optim, Tensor
from torch_sparse import SparseTensor, matmul

from torch_geometric.data import download_url, extract_zip
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.typing import Adj
from torch_geometric.nn.conv import MessagePassing

from scipy import sparse

In [9]:
r_mat = np.array([[0, 0, 1, 1],
                  [1, 0, 1, 0],
                  [0, 1, 0, 0],])
print(r_mat)

[[0 0 1 1]
 [1 0 1 0]
 [0 1 0 0]]


In [10]:
r_mat_np_coo = sparse.coo_matrix(r_mat)
print(r_mat_np_coo)


  (0, 2)	1
  (0, 3)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1


In [11]:
r_mat_edge_index = [r_mat_np_coo.row, r_mat_np_coo.col]


print(r_mat_edge_index[0])
print(r_mat_edge_index[1])

[array([0, 0, 1, 1, 2], dtype=int32), array([2, 3, 0, 2, 1], dtype=int32)]


In [ ]:
r_mat_np_coo.todense()

# Doing using Pytorch

In [ ]:
r_mat_edge_index = torch.LongTensor(np.array(r_mat_edge_index))

In [ ]:
sparse_r_mat_edge_index = SparseTensor(row=r_mat_edge_index[0], col=r_mat_edge_index[1], sparse_sizes=(3, 4))
print(sparse_r_mat_edge_index)

In [ ]:
dense_r_mat_edge_index = sparse_r_mat_edge_index.to_dense()
print(dense_r_mat_edge_index)   

In [ ]:
# In Bipartiite Graphs, the adjacency matrix is a block matrix with the shape of (N, M), where N and M are the number of nodes in the two partitions.
# in order to convert interaction matrix of a bipartite graph to adjacency matrix, we need to convert the interaction matrix to a block matrix with the shape of (N+M, N+M)

#      ( 0    R )
#  A = ( R_T  0 )

def convert_r_mat_edge_index_to_adj_mat_edge_index(input_edge_index: Tensor, row_size: int, col_size: int) -> Tensor:
    R = torch.zeros((row_size, col_size))
    
# convert sparse coo format to dense format to get R matrix
    for i in range(input_edge_index[0]):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = 1


# perform the r_mat to adj_mat conversion
    A = torch.zeros((row_size+col_size, row_size+col_size))
    A[:row_size, row_size:] = R
    A[row_size:, :row_size] = R.T

# convert from dense format to sparse coo format
    A_coo = torch.nonzero(A, as_tuple=False).T
    return A_coo

In [ ]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index: Tensor, row_size: int, col_size: int) -> Tensor:
    A = torch.zeros((row_size+col_size, row_size+col_size))
    for i in range(input_edge_index[0]):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        A[row_idx][col_idx] = 1

    R = A[:row_size, row_size:]
    R_coo = torch.nonzero(R, as_tuple=False).T
    return R_coo